In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from tqdm import tqdm
import random

from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.nasnet import preprocess_input
from glob import glob

from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

#         /kaggle/input/petfinder-pawpularity-score/test/
#         /kaggle/input/petfinder-pawpularity-score/train.csv
#         /kaggle/input/petfinder-pawpularity-score/test.csv
# /kaggle/input/petfinder-pawpularity-score/train/7fc71b8da143721939715b1cfe22122f.jpg

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

This notebook shows how a simple pretrained encoding of a pet image can connect similar images. The idea being that if you look like me then maybe you are as pawpular as me. 

In [ ]:
x_train = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
x_train.head()

In [ ]:
model = NASNetLarge(weights='imagenet', include_top=False, pooling='max')

img_path = '/kaggle/input/petfinder-pawpularity-score/train/7fc71b8da143721939715b1cfe22122f.jpg'
img = image.load_img(img_path, target_size=(331, 331))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)

In [ ]:
x_features = {}

paths = glob('/kaggle/input/petfinder-pawpularity-score/train/*jpg')
random.shuffle(paths)
                       
k = 9912 # change this for faster runtime
for i,img_file in enumerate(tqdm(paths)):
    # 10 it/s
    img = image.load_img(img_file, target_size=(331, 331))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    features = model.predict(x)
    x_features[img_file] = features[0]
    if i > k:
        break
    

In [ ]:
R = cosine_similarity(pd.DataFrame.from_dict(x_features, orient='index'))
np.fill_diagonal(R,0)
R = np.triu(R)

hits = np.argmax(R, axis=1)

file_list = list(x_features.keys())
dup_show = 5
shown = 0
for i,hit,file in zip(range(len(hits)), hits,file_list):
    if R[i,hit] > 0.95 and shown < dup_show: 
        fig = plt.figure()
        ax = fig.add_subplot(1, 2, 1)
        imgplot = plt.imshow(image.load_img(file, target_size=(331, 331)))
        ax.set_title('Seed')
        ax = fig.add_subplot(1, 2, 2)
        imgplot = plt.imshow(image.load_img(file_list[hit], target_size=(331, 331)))
        ax.set_title('Hit score:%g' % R[i, hit])
        shown += 1


In [ ]:
print('there are %g duplicates found' % (R > 0.95).sum())

The following histogram shows connection density within our database. How often do a pair of images look similar? What are some reasonable distances to give more or less similar pet photos? 

In [ ]:
pd.Series(R[np.triu_indices(len(R))]).hist()

In [ ]:
max_shows = 10
mid_hit_cnt = 0
upper_hit_cnt = 0
lower_hit_cnt = 0
poor_hit_cnt = 0

for i,hit,file in zip(range(len(hits)), hits,file_list):
    
    if R[i,hit] > 0.6 and R[i,hit] < 0.75 and mid_hit_cnt < max_shows: 
        fig = plt.figure()
        ax = fig.add_subplot(1, 2, 1)
        imgplot = plt.imshow(image.load_img(file, target_size=(331, 331)))
        ax.set_title('Seed')
        ax = fig.add_subplot(1, 2, 2)
        imgplot = plt.imshow(image.load_img(file_list[hit], target_size=(331, 331)))
        ax.set_title('Hit score:%g' % R[i, hit])
        mid_hit_cnt += 1
    if R[i,hit] > 0.75 and R[i,hit] < 0.95 and upper_hit_cnt < max_shows: 
        fig = plt.figure()
        ax = fig.add_subplot(1, 2, 1)
        imgplot = plt.imshow(image.load_img(file, target_size=(331, 331)))
        ax.set_title('Seed')
        ax = fig.add_subplot(1, 2, 2)
        imgplot = plt.imshow(image.load_img(file_list[hit], target_size=(331, 331)))
        ax.set_title('Hit score:%g' % R[i, hit])
        upper_hit_cnt += 1
    if R[i,hit] > 0.4 and R[i,hit] < 0.6 and lower_hit_cnt < max_shows: 
        fig = plt.figure()
        ax = fig.add_subplot(1, 2, 1)
        imgplot = plt.imshow(image.load_img(file, target_size=(331, 331)))
        ax.set_title('Seed')
        ax = fig.add_subplot(1, 2, 2)
        imgplot = plt.imshow(image.load_img(file_list[hit], target_size=(331, 331)))
        ax.set_title('Hit score:%g' % R[i, hit])
        lower_hit_cnt += 1
    if R[i,hit] < 0.4 and poor_hit_cnt < max_shows: 
        fig = plt.figure()
        ax = fig.add_subplot(1, 2, 1)
        imgplot = plt.imshow(image.load_img(file, target_size=(331, 331)))
        ax.set_title('Seed')
        ax = fig.add_subplot(1, 2, 2)
        imgplot = plt.imshow(image.load_img(file_list[hit], target_size=(331, 331)))
        ax.set_title('Hit score:%g' % R[i, hit])
        poor_hit_cnt += 1
    